In [1]:
import yaml

import utils
from utils.utils_units import conv_units
from utils.Plotting import _apply_log10_vals

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import cftime
from tqdm.auto import tqdm
from pathlib import Path
import dask
from dask_jobqueue import SLURMCluster
from distributed import Client, progress, wait
%matplotlib inline
%load_ext autoreload
%autoreload 2


In [2]:
cluster = SLURMCluster(cores=1, processes=1, memory="10GB")
cluster.adapt(minimum_jobs=34, maximum_jobs=35)
client = Client(cluster)
cluster

In [3]:
image_dir = Path("../images")
image_dir.mkdir(parents=True, exist_ok=True)

In [4]:
with open("diag_metadata.yaml", mode="r") as fptr:
    diag_metadata_list = yaml.safe_load(fptr)

In [5]:
def plot(da, diag_metadata, image_dir, cmap="plasma"):
    import matplotlib.pyplot as plt
    import numpy as np
    import xarray as xr

    plot_type = "timestep-global-map"

    try:
        time = str(da.time.data.item())
        varname = diag_metadata["varname"]
        for apply_log10 in _apply_log10_vals(diag_metadata):
            filename = f"{varname}+{time}"
            vmin = diag_metadata.get("map_vmin")
            vmax = diag_metadata.get("map_vmax")
            if apply_log10:
                if vmin is not None:
                    vmin = np.log10(vmin) if vmin > 0.0 else None
                if vmax is not None:
                    vmax = np.log10(vmax) if vmax > 0.0 else None
            to_plot = da.copy()

            if "display_units" in diag_metadata:
                to_plot = conv_units(to_plot, diag_metadata["display_units"])
            if apply_log10:
                to_plot = np.log10(xr.where(to_plot > 0.0, to_plot, np.nan))
                to_plot.name = f"log10({to_plot.name})"
                filename = f"{filename}+log_10@{apply_log10}"
            if "isel_dict" in diag_metadata:
                depth_levels = dict()
                for key in isel_dict:
                    depth_levels[key] = to_plot[key].data.item()
                s = "+".join(
                    [
                        "%s@%s" % (key, value)
                        for (key, value) in depth_levels.items()
                    ]
                )
                filename = f"{filename}+{s}.png"
            else:
                filename = f"{filename}.png"
            path = image_dir / plot_type / filename
            if path.exists():
                return da.time
            ax = to_plot.plot(cmap=cmap, vmin=vmin, vmax=vmax)
            fig = ax.get_figure()
            plt.savefig(path.as_posix(), dpi=300)
            plt.close(fig)
    except Exception as exc:
        print(exc)
    return da.time

In [6]:
case = utils.CaseClass(
    "g.e22.G1850ECO_JRA_HR.TL319_t13.004",
    start_date="0001-01",
    end_date="0004-12",
)
stream = "pop.h"
case._open_history_files(stream)

Datasets contain a total of 48 time samples
Last average written at 0005-01-01 00:00:00


In [7]:
plots, v = [], []
for diag_metadata in diag_metadata_list:
    z = dict(diag=diag_metadata.copy())
    varname = diag_metadata["varname"]
    isel_dict = diag_metadata.get("isel_dict")
    data = (
        case.history_contents[stream][varname]
        .isel(isel_dict)
        .chunk({"time": 1})
    )
    z["data"] = data
    template = xr.zeros_like(data.time).chunk({"time": 1})
    z["template"] = template
    p = xr.map_blocks(
        plot, data, args=[z["diag"], image_dir], template=template
    )
    v.append(z)
    plots.append(p)

In [8]:
plots = xr.concat(plots, dim="diag", compat="override", coords="minimal")
plots

<xarray.DataArray 'time' (diag: 23, time: 48)>
dask.array<concatenate, shape=(23, 48), dtype=object, chunksize=(1, 1), chunktype=numpy.ndarray>
Coordinates:
    z_t_150m  float32 dask.array<chunksize=(), meta=np.ndarray>
    z_t       float32 dask.array<chunksize=(), meta=np.ndarray>
  * time      (time) object 0001-01-16 12:15:50 ... 0004-12-16 12:00:00
Dimensions without coordinates: diag
Attributes:
    long_name:  time
    bounds:     time_bound

In [31]:
client.compute(plots, retries=2)

<Future: pending, key: finalize-3ee61c0fe262937b8bb9dd9a6dc8720c>